<a href="https://colab.research.google.com/github/jarusgnuj/asag/blob/main/ASAG_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# make sure you change runtime to gpu
!pip install simpletransformers

In [1]:
import json
import pandas as pd

In [9]:
from simpletransformers.classification import (
    ClassificationModel, ClassificationArgs
)

In [31]:
# method to read data into dataframes
def get_data(data_file):
  f = open(data_file)
  data_json = f.readlines()
  sent1=[]
  sent2=[]
  label=[]

  for data in data_json:
    t = json.loads(data)
    sent1.append(t['sentence1'])
    sent2.append(t['sentence2'])
    g_label = t['gold_label']
    if g_label == 'correct':
      label.append(1)
    elif g_label == 'incorrect':
      label.append(0)
    else:
      print("error")

  data_df = pd.DataFrame(list(zip(sent1, sent2, label)),
              columns=["text_a", "text_b", "labels"])
  return data_df

In [32]:
train_df = get_data('beetleTrain.jsonl')
test_df = get_data('beetleTest.jsonl')

In [33]:
train_df.head()

,text_a,text_b,labels
0,Terminal 1 and the positive terminal are separ...,positive battery terminal is separated by a ga...,1
1,Terminal 1 and the positive terminal are not c...,terminal 1 is not connected to the positive te...,1
2,Terminal 1 and the positive terminal are not c...,Because terminal 1 is connected to the positiv...,0
3,Terminal 1 and the positive terminal are separ...,because terminal 1 is not seperated by any gaps,0
4,Terminal 1 and the positive terminal are not c...,because terminal one is connected to both the...,0


In [34]:
test_df.head()

,text_a,text_b,labels
0,Terminal 4 and the positive terminal are separ...,positive battery terminal is separated by a ga...,1
1,Terminal 4 and the positive terminal are not c...,terminal 4 is not connected to the positive te...,1
2,Terminal 4 and the positive terminal are separ...,the positive battery terminal is seperated by ...,1
3,Terminal 4 and the positive terminal are not c...,the positive battery terminal is not connected...,1
4,Terminal 4 and the positive terminal are separ...,because terminal four and the positive termina...,0


In [40]:
# model arguments, for a quick test set epochs to 1. For a better model set to 10
model_args = ClassificationArgs(num_train_epochs=1)

# Create a ClassificationModel
model = ClassificationModel("roberta", "roberta-base")

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.bias', 'lm_head.bias', 'roberta.pooler.dense.bias', 'roberta.pooler.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.

In [15]:
# Train the model
model.train_model(train_df)

  0%|          | 0/3941 [00:00<?, ?it/s]

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/493 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:922: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default behavior will change in a future release to error out if a non-finite total norm is encountered. At that point, setting error_if_nonfinite=false will be required to retain the old behavior.
  model.parameters(), args.max_grad_norm


(493, 0.48038291870701383)

In [35]:
# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(
    test_df
)

  0%|          | 0/819 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/103 [00:00<?, ?it/s]

In [36]:
result

{'auprc': 0.7909236684216514,
 'auroc': 0.8584241126070992,
 'eval_loss': 0.5836525831407714,
 'fn': 100,
 'fp': 75,
 'mcc': 0.5581082228138882,
 'tn': 400,
 'tp': 244}

In [44]:
# load saved model
# the checkpoint file will be in the outputs directory
model = ClassificationModel(
    "roberta", "outputs/checkpoint-493-epoch-1"
)

In [45]:
# Make predictions with the model
predictions, raw_outputs = model.predict(
    [
        [
            "Terminal 1 and the positive terminal are separated by the gap",
            "terminal one was positive",
        ]
    ]
)
predictions

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

[0]

In [46]:
# correct label example
predictions, raw_outputs = model.predict(
    [
        [
            "Terminal 1 and the positive terminal are separated by the gap",
            "The positive battery terminal is separated by a gap from terminal 1",
        ]
    ]
)
predictions


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

[1]